In [257]:
import cv2
import numpy as np

In [258]:
def display(image, scale=0.5):
    shrunk = cv2.resize(image, (0,0), fx=scale, fy=scale)
    cv2.imshow("Screen Location", shrunk)

    if cv2.waitKey(1000) & 0xFF == ord('q'):
        cv2.destroyAllWindows()

In [259]:
billboard_colour = [150, 20, 20]
def strip_colours(image, preserve=billboard_colour):
    black_indices = np.where((image != preserve).all(axis=2))
    white_indices = np.where((image == preserve).all(axis=2))
    rtn = image.copy()
    rtn[black_indices] = [0,0,0]
    rtn[white_indices] = [255,255,255]
    return rtn

# noinspection PyDefaultArgument
def add_borders(image, border_color= [0,0,0], border_width=25):
    return cv2.copyMakeBorder(
        image,
        border_width,
        border_width,
        border_width,
        border_width,
        cv2.BORDER_CONSTANT,
        value=border_color
    )

In [260]:
source_image = cv2.imread("../resources/new/015_00010.png")
print(source_image.shape)
padded_source_image = add_borders(source_image)

(1080, 1920, 3)


In [261]:
binary_image = strip_colours(padded_source_image)

In [262]:
edged_image = cv2.Canny(binary_image, 30, 200)

(contours, _) = cv2.findContours(
    edged_image.copy(),
    cv2.RETR_TREE,
    cv2.CHAIN_APPROX_SIMPLE
)
contours = sorted(
    contours,
    key=cv2.contourArea,
    reverse=True
)[:10]


best = (0,0)

for contour in contours:
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
    area = cv2.contourArea(contour)
    print(area, end="\t")
    print(len(approx))
    # if len(approx) == 4:
    if area > best[0]:
      best = (area, approx)

print(best[1])

37642.5	4
37641.5	4
2413.5	4
2406.5	4
100.5	8
100.0	8
16.0	12
13.0	11
10.0	6
10.0	8
[[[1944  136]]

 [[1674  181]]

 [[1666  307]]

 [[1944  283]]]


In [263]:
display_image = padded_source_image.copy()
# Take the best (largest) contour
contours = best[1]

# contours = np.subtract(contours, )

cv2.drawContours(
    display_image,
    [best[1]],
    -1,
    (0,255,0),
    3
)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [264]:
# crop out border
display_image = display_image[25:-25, 25:-25]

In [265]:
display(source_image)
display(display_image)